# Introduction and Overview to Using LIGO/Virgo GW-strain data

These imports are more to check that you have the correct packages installed in your environment for use in this tutorial.

In [2]:
%matplotlib inline
import gwpy
import gwosc
import pycbc
import lalinference

## Querying for event information

The gwosc.datasets module provides tools to search for datasets, including filtering on GPS times.

For example, we can search for what event datasets are available:

In [6]:
from gwosc.datasets import find_datasets
events = find_datasets(type="event", detector="H1", segment=(1126051217, 1137254417))
print(events)

['151008', '151012A', '151116', 'GW150914', 'GW151012', 'GW151226']


If we want to find the gps times for different science operations we can use the `run_segment` method.

In [7]:
from gwosc.datasets import run_segment
print(run_segment('O1'))

(1126051217, 1137254417)


Furthermore to access data from different events that are recorded we can find the urls where to download the data such for the first event.

In [8]:
from gwosc.locate import get_event_urls
urls = get_event_urls('GW150914') # detector = , duration = )
print(urls)

[u'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5', u'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5', u'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5', u'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126257415-4096.hdf5']


# Basic data querying exercises

- How long did O2 last, in human-readable format, i.e. (days, months, years)?
- How many events were detected during O1?
- What file URL contains data for V1 4096 seconds around GW170817?

Now let's see what GW strain data looks like. We will again use the gwosc package to get the data.

In [ ]:
from gwosc.datasets import event_gps
gps = event_gps('GW170817')
print(gps)

In [ ]:
segment = (int(gps)-5, int(gps)+5)
print(segment)

In [ ]:
from gwpy.timeseries import TimeSeries
hdata = TimeSeries.fetch_open_data('H1', *segment, verbose=True)
print(hdata)

The TimeSeries class comes with a load of useful methods for doing basic operations with the GW-strain data, particularly plotting.

In [ ]:
plot = hdata.plot()

# Download and plot data

- Using the methods discussed in parts one and two, download the data from the Livingston detector for `GW150914` and plot the strain for a segement of 30 seconds around the merger.

Now that we have some data, let's use the data from `GW150914` to begin interrogating the properties of the data

In [ ]:
# Set the data form the Livingston detect or ldata
ldata = 

For starters, let's take a basic fft of the data and see what we get.

In [ ]:
fft = ldata.fft()
print(fft)

We see that the above object is no longer an instance of our TimeSeries object that we originally read the data in as. It is now a FrequencySeries object, related, but with different properties and methods.

In [ ]:
plot = fft.abs().plot(xscale="log", yscale="log")
plot.show()

Although this is the amplitude spectral density, it doesn't look terribly much like the typical LIGO/Virgo like sensitivity plots that we see. This is because we need to window and bandpass the data. However, instead of doing all these steps separately, the TimeSeries and Frequency series objects have methods built-in to do this.

In [ ]:
asd = ldata.asd(fftlength=4, method="median")
plot = asd.plot()
ax = plot.gca()
ax.set_xlim(10, 1400)
ax.set_ylim(2e-24, 1e-20)
plot.show()

However, with such short data lengths, there is a lot of noise in the ASD. Given a longer set of data, can you plot the ASD around a different event?

# ASD Exercise

- Obtain data from a different event, let's say `GW170817` and plot the ASD for both the Hanford and Livingston detectors. Get a section of data longer than 120s.

The code below will take care of the plotting for you, please fill in teh steps to retrieve the data and perform the ASD.

In [ ]:
gps = 
# get Hanford data
hdata2 = 
hasd2 = 
# get Livingston data
ldata2 = 
lasd2 = 

In [ ]:
plot = hasd2.plot()
ax = plot.gca()
ax.set_xlim(10, 1400)
ax.set_ylim(5e-24, 1e-20)
# and plot using standard colours
ax.plot(lasd2, label='LIGO-Livingston', color='gwpy:ligo-livingston')
# update the Hanford line to use standard colour, and have a label
hline = ax.lines[0]
hline.set_color('gwpy:ligo-hanford')  # change colour of Hanford data
hline.set_label('LIGO-Hanford')

ax.set_ylabel(r'Strain noise [$1/\sqrt{\mathrm{Hz}}$]')
ax.legend()
plot.show()

Another, very interesting way to interrogate GW data is too look at the famous Q-transform plots which effectively show PSD evolution with time. Using the data we already have let's look at one of these 'spectrograms' which is a more basic form of the Q-plot.

In [ ]:
specgram = ldata.spectrogram2(fftlength=4, overlap=2, window='hann') ** (1/2.)
plot = specgram.plot()
ax = plot.gca()
ax.set_yscale('log')
ax.set_ylim(10, 1400)
ax.colorbar(
    clim=(1e-24, 1e-20),
    norm="log",
    label=r"Strain noise [$1/\sqrt{\mathrm{Hz}}$]",
)
plot.show()  # refresh

This looks fine, but it doesn't show the nice 'tracks' that became some famous with the first detection `GW150914` and `GW170817`. This requires a Q-transform plot. With getting into the details, a Q-transform plot essentially finds the optimal window size for each time which returns the highest Q value of that PSD. This allows us to pick out the highest power features as they sweep through frequency space while evolving with time.

In [ ]:
gps = event_gps('GW170817')
segment = (int(gps) - 30, int(gps) + 2)
hdata = TimeSeries.fetch_open_data('H1', *segment, verbose=True, cache=True)

Thankfully this q-transform is another hand method of the GWpy TimeSeries object, so all we need to do is:

In [ ]:
hq = hdata.q_transform(frange=(30, 500))
plot = hq.plot()
plot.colorbar(label="Normalised energy")

You can almost see a track in the data. However, we can optimze our choice of the range of allowed q to visualize this better.

In [ ]:
hq = hdata.q_transform(frange=(30, 500), qrange=(100, 110))
plot = hq.plot()
ax = plot.gca()
ax.set_epoch(gps)
ax.set_yscale('log')
ax.colorbar(label="Normalised energy")

# Q-transform exercise

- Can you repeat the above for `GW170817` but for the Livngston detector? What do you see?
- Can you modify the tranform or the visualization to better see the inspiral in the data?